In [ ]:
import joblib
import os
import joblib
from sklearn.metrics import f1_score, classification_report
import pandas as pd

## Train and save the latest KNN, Decision Tree, and MLP models 

In [ ]:
%run data_preprocessing.ipynb
%run train_evaluate.ipynb

## Find the model with the best validation F1 score

In [ ]:
best_model = max(model_avg_val_f1, key=model_avg_val_f1.get)
print(f"Best model is the {best_model} model, with an average validation F1 score of {model_avg_val_f1[best_model]}.")

## Load the best model and run it on the test data

In [ ]:
os.makedirs("results", exist_ok=True)

if best_model == "KNN" or best_model == "Decision Tree":
    fname = f"{best_model.lower().replace(' ', '_')}.pkl"
    path  = os.path.join("models", fname)

    # # load it
    model = joblib.load(path)

    # # run on your test set
    y_pred = model.predict(X_test)

    pd.DataFrame({"prediction": y_pred}) \
      .to_csv(f"results/result.csv", index=False)

if best_model == "MLP":
    model = MLP(INPUT_DIM).to(device)
    state = torch.load(f"{save_dir}/MLP.pth", weights_only=True)
    model.load_state_dict(state)
    model.eval()

    X_tensor = torch.from_numpy(X_test_transformed).float().to(device)
    with torch.no_grad():
        probs = torch.sigmoid(model(X_tensor)).cpu().numpy()
        preds  = (probs > 0.5).astype(int)             # 0/1

    pd.DataFrame({"prediction": preds}).to_csv("results/result.csv", index=False)
    print("→ Saved result.csv with", len(preds), "rows")
